In [8]:
import pandas as pd
import numpy as np
import os
import requests
import json


In [9]:
def data_import():
    """
    Import der Daten aus allen Dateien, die mit 'waqi-covid-' anfangen.
    Entfernen von Duplikaten und Umbenennung bestimmter Spaltenwerte.
    Zusammenführung der DataFrames.
    """
    data_folder = './data/'
    all_files = [f for f in os.listdir(data_folder) if f.startswith('waqi-covid-') and f.endswith('.csv') or f == 'airquality-covid19-cities.json']
    dataframes = []

    if not all_files:
        print("Keine Dateien gefunden.")
        return None

    for file in all_files:
        file_path = os.path.join(data_folder, file)
        try:
            df = pd.read_csv(file_path, comment='#')

            if "Specie" not in df.columns:
                print(f"Spalte 'Specie' fehlt in {file}")
                continue

            df["Specie"] = df["Specie"].replace("wind gust", "wind-gust")
            df["Specie"] = df["Specie"].replace("wind speed", "wind-speed")
            df = df.drop_duplicates()

            if df.empty:
                print(f"{file} enthält nach Duplikat-Entfernung keine Daten mehr.")
                continue

            dataframes.append(df)
        except Exception as e:
            print(f"Fehler beim Verarbeiten von {file}: {e}")

    if not dataframes:
        print("Keine gültigen Daten vorhanden.")
        return None

    return pd.concat(dataframes, ignore_index=True)

In [10]:
df=data_import()
print(df)

Spalte 'Specie' fehlt in airquality-covid19-cities.json
waqi-covid-2022Q1.csv enthält nach Duplikat-Entfernung keine Daten mehr.
                Date Country    City Specie  count   min    max  median  \
0         2015-01-06      KR  Jeonju     co    124   0.1   12.3     4.5   
1         2015-01-22      KR  Jeonju     co    116   4.5   10.0     6.7   
2         2015-03-30      KR  Jeonju     co    118   1.2   11.2     5.6   
3         2015-05-27      KR  Jeonju     co     93   2.3    5.6     3.4   
4         2015-02-03      KR  Jeonju     co    133   4.5   13.4     7.8   
...              ...     ...     ...    ...    ...   ...    ...     ...   
14251935  2024-08-12      IR  Tehran    so2    154   6.0  101.0    12.0   
14251936  2025-01-13      IR  Tehran    so2    516  10.0   74.0    19.0   
14251937  2024-02-10      IR  Tehran    so2    299   5.0   76.0    34.0   
14251938  2024-05-16      IR  Tehran    so2    559   6.0   92.0    17.0   
14251939  2025-01-04      IR  Tehran    so2   

In [25]:
def geo_data(df):
    """
    Fügt die Geodaten zu den Städten hinzu
    """
    df = df.copy()

    # JSON-Datei laden mit Fehlerbehandlung
    try:
        with open('./data/airquality-covid19-cities.json', 'r', encoding='utf-8') as file:
            geodata = json.load(file)
    except (FileNotFoundError, json.JSONDecodeError) as e:
        print(f"Fehler beim Laden der JSON-Datei: {e}")
        return df

    geodata = geodata.get("data", [])

    # Erstellen eines DataFrames mit Städten und Geokoordinaten
    df_places = pd.DataFrame([
        {
            "City": entry.get("Place", {}).get("name"),
            "Latitude": entry.get("Place", {}).get("geo", [None, None])[0],
            "Longitude": entry.get("Place", {}).get("geo", [None, None])[1]
        }
        for entry in geodata if "Place" in entry and "geo" in entry.get("Place", {})
    ])

    # Entferne Zeilen mit fehlenden Stadtnamen
    df_places.dropna(subset=["City"], inplace=True)

    # Standardisiere Stadtnamen
    df_places["City"] = df_places["City"].str.lower().str.strip()
    df["City"] = df["City"].str.lower().str.strip()

    # Zusammenführen der beiden DataFrames über "City"
    df = df.merge(df_places, on="City", how="left")

    # Überprüfung auf fehlende Geodaten
    fehlende_staedte = df[df["Latitude"].isna()]["City"].unique()
    if fehlende_staedte.size > 0:
        print(f"Keine Geodaten gefunden für: {', '.join(fehlende_staedte)}")

    return df

In [ ]:
df = geo_data(df)
print(df.sample(10))

Keine Geodaten gefunden für: washington d.c.
                Date Country       City       Specie  count     min     max  \
3197051   2019-09-03      IN  ghāziābād          dew     44    24.0    28.0   
6038811   2020-12-21      JP   yokohama          no2    547     1.9    54.4   
188512    2015-05-31      FR   besançon         pm10     48     3.0    22.0   
11370018  2023-05-19      FR      rouen    wind-gust     20     0.3     6.5   
6646049   2021-03-11      TR      adana  temperature     96     8.0    22.5   
11712506  2023-12-22      FI   helsinki    wind-gust     67     0.1    18.3   
11358610  2023-06-03      FR     nantes  temperature     48    12.5    26.0   
6931822   2021-05-05      RO      bacău  temperature     48     7.6    19.9   
2594630   2019-05-10      RO      bacău     pressure     48  1005.0  1010.0   
3558533   2019-12-10      VN        huế     pressure     24  1016.0  1020.0   

          median  variance  Latitude  Longitude  
3197051     26.0      9.92  28.6653

In [23]:
def data_cleaning(df):
    """Bereinigung der Daten
    - nicht benötigte Spalten löschen
    - Zusammenfassung der Daten nach Datum, Land, Stadt und Spezies, so dass nur ein Messwert je Species (Median) pro Tag/ Stadt verbleibt
    - Spalte Species aufteilen
    - df als csv speichern im Datenverzeichnis
    """
    df = df.copy()

    df = df.drop(columns=['variance', 'min', 'max'], errors='ignore')

    df = df.groupby(["Date", "Country", "City", "Specie"], as_index=False).agg({"median": "mean"})  

    df = df.pivot(index=["Date", "Country", "City"], columns="Specie", values='median').reset_index()

    df["City"] = df["City"].str.lower().str.strip()
    df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d')

    df = geo_data(df)

    df = weather_data(df)

    output_path = './data/cleaned_data.csv'
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    df.to_csv(output_path, index=False)
    print(f"✅ Datei wurde gespeichert: {output_path}")

    return df



In [22]:
from meteostat import Daily, Stations
from datetime import datetime
import pandas as pd

def weather_data(df):
    """
    Ruft Wetterdaten für Städte im DataFrame ab und integriert sie.
    """
    df = df.copy()

    # Städte extrahieren und Duplikate entfernen
    cities = df[['City', 'Latitude', 'Longitude']].drop_duplicates()

    # Zeitspanne festlegen
    start = datetime(2015, 1, 1)
    end = datetime(2024, 12, 31)

    # DataFrame für alle Städte vorbereiten
    all_data = pd.DataFrame()

    # Wetterdaten für jede Stadt abrufen und hinzufügen
    for _, city in cities.iterrows():
        try:
            # Nächste Wetterstation suchen
            stations = Stations().nearby(city['Latitude'], city['Longitude'])
            station = stations.fetch(1)

            if not station.empty:
                station_id = station.index[0]

                # Tägliche Wetterdaten abrufen
                data = Daily(station_id, start, end).fetch()

                # NaN-Daten rausfiltern
                data.dropna(how='all', inplace=True)

                if not data.empty:
                    # Stadtname hinzufügen
                    data["City"] = city["City"]

                    # Daten in den Gesamtdaten-Frame einfügen
                    all_data = pd.concat([all_data, data])

        except Exception as e:
            print(f"⚠️ Fehler beim Abrufen der Daten für {city['City']}: {e}")

    # Index zurücksetzen
    all_data.reset_index(inplace=True)

    # Spalte 'time' umbenennen in 'Date'
    all_data.rename(columns={'time': 'Date'}, inplace=True)

    # Standardisiere den Stadtnamen
    all_data["City"] = all_data["City"].str.lower().str.strip()

    print(f"✅ Wetterdaten gesammelt für {all_data['City'].nunique()} Städte")

    # Konvertiere die 'Date'-Spalte in beiden DataFrames zu String-Format
    # df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d')
    all_data['Date'] = pd.to_datetime(all_data['Date']).dt.strftime('%Y-%m-%d')

    # Berechne den Anteil der NaN-Werte pro Spalte
    missing_percentage = all_data.isna().mean() * 100
    # Lösche Spalten mit mehr als 80% NaN-Werten
    all_data = all_data.loc[:, missing_percentage <= 80]

    # Zusammenführen der beiden DataFrames über "City" und "Date"
    df = pd.merge(df, all_data, on=["City", 'Date'], how="left")

    return df


In [28]:
#df = data_cleaning(df)
print(df.sample(10).T)

                  1195618     775112      487279      869486      1395905  \
Date           2023-02-14  2020-12-11  2019-08-24  2021-05-16  2024-05-07   
Country                JP          FR          XK          FR          CN   
City                 kobe        metz    pristina       dijon      hegang   
aqi                   NaN         NaN         NaN         NaN         NaN   
co                    3.4         NaN         NaN         NaN         4.6   
d                     NaN         NaN         NaN         NaN         NaN   
dew                  -3.0         2.0         NaN         9.0        -0.4   
humidity             56.0        91.0        51.9        87.0        32.8   
mepaqi                NaN         NaN         NaN         NaN         NaN   
neph                  NaN         NaN         NaN         NaN         NaN   
no2                   3.8        10.1         NaN         3.1         3.7   
o3                   32.0         7.8         NaN        25.3        34.6   

In [ ]:
#WEtterdaten integrieren


from meteostat import Daily, Stations
from datetime import datetime

df = df_copy()

# Städte
cities = df[['City', 'Latitude', 'Longitude']].drop_duplicates()

# Zeitspanne festlegen
start = datetime(2015, 1, 1)
end = datetime(2024, 12, 31)

# DataFrame für alle Städte vorbereiten
all_data = pd.DataFrame()

# Daten für jede Stadt abrufen und hinzufügen
for _, city in cities.iterrows():
    # Nächste Wetterstation suchen
    stations = Stations().nearby(city['Latitude'], city['Longitude'])
    station = stations.fetch(1)

    if not station.empty:
        station_id = station.index[0]

        # Tägliche Wetterdaten abrufen
        data = Daily(station_id, start, end).fetch()

        # Nan-Daten rausfiltern
        data.dropna(how='all', inplace=True)

        if not data.empty:
            # Stadtname hinzufügen
            data["City"] = city["City"]

            # Daten in den Gesamtdaten-Frame einfügen
            all_data = pd.concat([all_data, data])


# Index zurücksetzen
all_data.reset_index(inplace=True)

# Spalte time umbennen in Date
all_data.rename(columns={'time': 'Date'}, inplace=True) 

# Standardisiere den Stadtnamen für eine bessere Übereinstimmung
all_data["City"] = all_data["City"].str.lower().str.strip()

print(f"✅ Wetterdaten gesammelt für {all_data['City'].nunique()} Städte")

# # Ergebnis anzeigen
# print(all_data.head())

# Konvertiere die 'Date' Spalte in beiden DataFrames zu datetime
df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d')
all_data['Date'] = pd.to_datetime(all_data['Date']).dt.strftime('%Y-%m-%d')

# Berechne den Anteil der NaN-Werte pro Spalte
missing_percentage = all_data.isna().mean() * 100

# Lösche die Spalten, bei denen der Anteil an NaN-Werten größer als 80% ist
all_data = all_data.loc[:, missing_percentage <= 80]


# Zusammenführen der beiden DataFrames über "City"
df = pd.merge(df, all_data, on=["City", 'Date'],how="left")

In [ ]:

print(df.shape)
df.sample(30)